# CONAN

In [ ]:
from collections import Counter
import datetime
import gzip
import pathlib
import os
import plotly.graph_objects as go
import kaleido

import matplotlib.pyplot as plt
import numpy
import scipy
from scipy import ndimage

%matplotlib inline

class PearsonAnalysis:
    """Performs peak//valley analysis of Pearson-time data.
    
    Parameters
    ----------
    file_path: str
        Path to the image data.
    z_threshold: float
        Flat cutoff for data in z dimension.
    x_width_nm: float
        Width of x dimension of image in nanometres.
    """
    
    def __init__(self, file_path):
        print('Loading data from "{}"...'.format(file_path))
        start_time = datetime.datetime.now()
        
        # Pre-processing of peaks and valleys data to get all_data and all_clustered_data
        self.raw_data = self.load_CONAN_as_2d_array(file_path) # Was previously: self.raw_data = numpy.reshape(raw_data_col, (-1, 4096))[::-1]
        self.nresidues = len(self.raw_data)
        self.pea_data = self.raw_data > 0
        self.vly_data = self.raw_data < 0
        self.pea_clustered_data, self.pea_cluster_num = scipy.ndimage.measurements.label(self.pea_data) # Generates an array of indexes on the features
        self.vly_clustered_data, self.vly_cluster_num = scipy.ndimage.measurements.label(self.vly_data) # The clustered_data are input for several fucntions
        self.vly_clustered_data = -self.vly_clustered_data
        self.all_cluster_num = self.pea_cluster_num + self.vly_cluster_num
        zip_clustered_data = numpy.dstack((self.pea_clustered_data, self.vly_clustered_data))
        self.all_clustered_data = numpy.zeros((self.nresidues,self.nresidues))
        for row in range(0,len(self.all_clustered_data)):
            for column in range(0,len(self.all_clustered_data[row])):
                self.all_clustered_data[row][column] = sum(zip_clustered_data[row][column])
        
        end_time = datetime.datetime.now()
        time_taken = end_time - start_time
        print("Data loaded in {} seconds.".format(time_taken.seconds))

    def __len__(self):
        return self.all_cluster_num

    def get_cluster_pixel_counts(self, min_pixel_count=0, max_pixel_count=None):
        """Generates a dictionary containing pixel counts for each cluster.
        
        Parameters
        ----------
        min_pixel_count: int
            Minimum number of pixels in cluster.
        max_pixel_count: int
            Maximum number of pixels in cluster.
        
        Returns
        -------
        cluster_pixel_dict: dict
            Keys are the cluster identifier and values are the corresponding
            pixel count.
        """
        cluster_pixel_dict = Counter(self.all_clustered_data.flatten())
        del cluster_pixel_dict[0]
        if max_pixel_count is None:
            max_pixel_count = max(cluster_pixel_dict.values())
        return dict(
            filter(
                lambda x: min_pixel_count <= x[1] <= max_pixel_count,
                cluster_pixel_dict.items(),
            )
        )
    
    def get_cluster_heights(self, min_height=None, max_height=None):
        """Generates a dictionary containing height (Pearson coefficient) information for each cluster.

        Parameters
        ----------
        min_height: float
            Minimum height of cluster.
        max_height: float
            Maximum height of cluster.

        Returns
        -------
        cluster_height_dict: dict
            Keys are the cluster identifier and values are the corresponding
            height information in the format (mean, min, max, stddev, median).
        """
        if max_height is None:
            max_height = numpy.max(self.raw_data)
        if min_height is None:
            min_height = numpy.min(self.raw_data)
        zipped_matrices = numpy.dstack((self.raw_data, self.all_clustered_data)) # dstack will "stack" two arrays on top of each other. Creates a 370x370 matrix where each point is a list of two values: Pearson and cluster 
        cluster_height_dict = {}
        for row in zipped_matrices:
            for (height, label) in row: # Each height, label pair corresponds to a residue pair
                if min_height <= height <= max_height:
                    label = int(label)
                    if label:
                        if label not in cluster_height_dict: # Make a new list for each cluster...
                            cluster_height_dict[label] = []
                        cluster_height_dict[label].append(height)# ... containing the Pearson values of all points within the cluster
        return {
            k: (numpy.mean(v), min(v), max(v), numpy.std(v), numpy.median(v))
            for k, v in cluster_height_dict.items()
        } # Then for each key only return the interesting data about the cluster
    
    def clustered_filtered_by_pixel_count(self, min_pixel_count, max_pixel_count=None):
        """Filters the cluster array by pixel count.

        Parameters
        ----------
        min_pixel_count: int
            Minimum number of pixels in cluster.
        max_pixel_count: int
            Maximum number of pixels in cluster.

        Returns
        -------
        filtered_cluster_data: numpy.array
            Updated cluster array (with cluster idices) removing clusters outside cut-off values.
        """
        pixel_dict = self.get_cluster_pixel_counts(min_pixel_count, max_pixel_count) # This gives a pixel_dict with given cutoffs
        
        def if_in_pixel_dict(x):
            if x in pixel_dict.keys(): # The vectorised function goes through each point in the clustered data. If the value is in the pixel_dict keys, then it keeps that value. If not, it replaces it ith zero.
                return x
            else:
                return 0
        
        v_if_in_pixel_dict = numpy.vectorize(if_in_pixel_dict)
        return v_if_in_pixel_dict(self.all_clustered_data)
    
    def get_maxmin_cluster_heights(self, min_height=None, max_height=None): 
        """Generates a dictionary containing height (Pearson coefficient) information for each peak and valley.

        Parameters
        ----------
        min_height: float
            Minimum height of cluster.
        max_height: float
            Maximum height of cluster.

        Returns
        -------
        cluster_height_dict: dict
            Keys are the cluster identifier and values are the max for peaks and min for valleys.
        """
        if max_height is None:
            max_height = numpy.max(self.raw_data)
        if min_height is None:
            min_height = numpy.min(self.raw_data)
        zipped_matrices = numpy.dstack((self.raw_data, self.all_clustered_data))
        cluster_height_dict = {}
        for row in zipped_matrices:
            for (height, label) in row:
                label = int(label)
                if label:
                    if label not in cluster_height_dict:
                        cluster_height_dict[label] = []
                    cluster_height_dict[label].append(height)
        cluster_maxmin_height_dict = {}
        for k, v in cluster_height_dict.items(): # Get the max or min values at the end of the value list.
            if k > 0: # For peaks get the max value
                cluster_maxmin_height_dict[k] = (max(v))
            if k < 0: # For valleys get the min value
                cluster_maxmin_height_dict[k] = (min(v))
        return cluster_maxmin_height_dict
    
#    def clustered_filtered_by_magnitude(self, valley_cutoff, peak_cutoff):
#        """Filters the cluster array by pixel count.#

#        Parameters
#        ----------
#        min_pixel_count: int
#            Minimum number of pixels in cluster.
#        max_pixel_count: int
#            Maximum number of pixels in cluster.##

#        Returns
#        -------
#        filtered_cluster_data: numpy.array
#            Updated cluster array (with cluster idices) removing clusters outside cut-off values.
#        """
#        maxmin_dict = self.get_maxmin_cluster_heights(min_pixel_count, max_pixel_count) # This gives a pixel_dict with given cutoffs
#    
#maxmin_dict = self.get_maxmin_cluster_heights()
#maxmin_filtered_dict = dict(
#            filter(
#                lambda x: 0.8 <= abs(x[1]) <= 1,
#                maxmin_dict.items(),
#            )
#    )
    
    def array_from_clustered_dict(self, clustered_dict):
        """
        Creates a numpy array from a dictionary of cluster indices.
        Returns an arra of raw Pearson data values far all points of these clusters.
        """
        
        array = numpy.zeros((self.nresidues,self.nresidues))
        for row in range(0,len(array)):
            for col in range(0,len(array)):
                if self.all_clustered_data[row][col] in clustered_dict.keys():
                    array[row][col] = self.raw_data[row][col]
        return array
    
    def merge_cluster_dicts(major_dict, minor_dict):
        merged_dict = {}
        for k, v in major_dict.items(): # for keys, values in items of major dict...
            merged_dict[k] = (v, minor_dict[k]) # merged dict has the value of the major dict AND the minor dict, in form k : (
        return merged_dict

    @staticmethod
    def load_CONAN_as_2d_array(file_path):
        """Creates a numpy array from a CONAN Pearson-time data file."""
        with open(file_path, "r") as inf:
            #array = None # Need to input how many residues there are
            number_of_residues = None
            for line in inf.readlines()[::-1]: # Go from last residue first, so that the 1,1 point is in the bottom left
                try:
                    row, column, pear = line.split() # Get the columns
                    if not number_of_residues:
                        number_of_residues = int(row) # Get the index of the final residue, use it to define the dimensions of raw_data
                        data = numpy.zeros((number_of_residues,number_of_residues))
                    data[int(row)-1][int(column)-1] = float(pear) # Put the value in the correct place
                except:
                    pass # Pass empty rows that would cause error
        return data # Get the 1,1 index at the bottom left

In [ ]:
def conan_graph(inpPA, title="", meanP=0, medP=0, mxmnP=0, stdvP=0, res_interest=[], save=0):
    pa1 = PearsonAnalysis(inpPA)
    
    # Plot matrix per cluster so each cluster can be labelled with the folowing hover-information:
    # peak/valley max/min percentile, peak/valley mean percentile, peak/valley median percentile, stdev percentile
    
    # Get a dictionary with k = cluster and v = percentile ranks + list of residues

    cluster_info_dict = pa1.get_cluster_heights() # clust: (numpy.mean(v), min(v), max(v), numpy.std(v), numpy.median(v))
    maxmin_dict = pa1.get_maxmin_cluster_heights() # clust: max if peak or min if valley
    cent_dict = {}

    for clust in cluster_info_dict.keys():
        cent_dict[clust] = [] # clust: [mean, median, max/min, std]

    # Get mean percentile
    # Get ranking of indices
    pea_rnk = sorted([v[0] for v in cluster_info_dict.values() if v[0] > 0], reverse=True) # Must be reversed so that index() returns the highest index of the queried value
    vly_rnk = sorted([v[0] for v in cluster_info_dict.values() if v[0] < 0], reverse=False)

    # For each cluster, find how many values it is greater than or equal to
    for clust, v in cluster_info_dict.items():
        if clust > 0:
            cent_dict[clust].append(round((len(pea_rnk)-pea_rnk.index(v[0])) / len(pea_rnk) * 100, 2))
        if clust < 0:
            cent_dict[clust].append(round((len(vly_rnk)-vly_rnk.index(v[0])) / len(vly_rnk) * 100, 2))

    # Get median percentile
    # Get ranking of indices
    pea_rnk = sorted([v[4] for v in cluster_info_dict.values() if v[4] > 0], reverse=True) # Must be reversed so that index() returns the highest index of the queried value
    vly_rnk = sorted([v[4] for v in cluster_info_dict.values() if v[4] < 0], reverse=False)

    # For each cluster, find how many values it is greater than or equal to
    for clust, v in cluster_info_dict.items():
        if clust > 0:
            cent_dict[clust].append(round((len(pea_rnk)-pea_rnk.index(v[4])) / len(pea_rnk) * 100, 2))
        if clust < 0:
            cent_dict[clust].append(round((len(vly_rnk)-vly_rnk.index(v[4])) / len(vly_rnk) * 100, 2))

    # Get max/min percentile
    # Get ranking of indices
    pea_rnk = sorted([v for v in maxmin_dict.values() if v > 0], reverse=True) # Must be reversed so that index() returns the highest index of the queried value
    vly_rnk = sorted([v for v in maxmin_dict.values() if v < 0], reverse=False)

    # For each cluster, find how many values it is greater than or equal to
    for clust, v in maxmin_dict.items():
        if clust > 0:
            cent_dict[clust].append(round((len(pea_rnk)-pea_rnk.index(v)) / len(pea_rnk) * 100, 2))
        if clust < 0:
            cent_dict[clust].append(round((len(vly_rnk)-vly_rnk.index(v)) / len(vly_rnk) * 100, 2))

    # Get stdev percentile
    # Get ranking of indices
    rnk = sorted([v[3] for v in cluster_info_dict.values()], reverse=True) # Must be reversed so that index() returns the highest index of the queried value  

    # For each cluster, find how many values it is greater than or equal to
    for clust, v in cluster_info_dict.items():
        cent_dict[clust].append(round((len(rnk) - rnk.index(v[3])) / len(rnk) * 100,2))

    # make percentile arrays
    cluster_dict = pa1.all_clustered_data
    mean_percent_array = numpy.zeros((370,370))
    median_percent_array = numpy.zeros((370,370))
    maxmin_percent_array = numpy.zeros((370,370))
    stdev_percent_array = numpy.zeros((370,370))
    for row in range(0, len(cluster_dict)):
        for col in range(0, len(cluster_dict[row])):
            if cluster_dict[row][col]:
                mean_percent_array[row][col] = cent_dict[cluster_dict[row][col]][0]
                median_percent_array[row][col] = cent_dict[cluster_dict[row][col]][1]
                maxmin_percent_array[row][col] = cent_dict[cluster_dict[row][col]][2]
                stdev_percent_array[row][col] = cent_dict[cluster_dict[row][col]][3]
    
    # Filter by percentile. > Nth for all metrics
    filt_by_cent = numpy.zeros((370,370))
    percent_arrays = [mean_percent_array, median_percent_array, maxmin_percent_array, stdev_percent_array]
    for row in range(0, len(filt_by_cent)):
        for col in range(0, len(filt_by_cent[row])):
            if mean_percent_array[row][col] > meanP:
                if median_percent_array[row][col] > medP:
                    if maxmin_percent_array[row][col] > mxmnP:
                        if stdev_percent_array[row][col] > stdvP:
                            filt_by_cent[row][col] = pa1.raw_data[row][col]

    fig = go.Figure()
    fig.select_coloraxes
    fig.add_trace(go.Heatmap(
        z=filt_by_cent,
        colorscale="PuOr",
        customdata=numpy.moveaxis([mean_percent_array, median_percent_array, maxmin_percent_array, stdev_percent_array], 0,-1),
        hovertemplate="   mean %ile: %{customdata[0]}<br>   median %ile: %{customdata[1]}<br>   maxmin %ile: %{customdata[2]}<br>   stdev %ile: %{customdata[3]}"
        )
             )
    #res_interest = [170,169,171,335,348,346] # This defines the RANKING of the lines 
    fig.add_trace(go.Bar(
        x=sorted(res_interest), # This must be sorted, otherwise the heatmap of the bars will be done by ascending order inserted of the RANKING
        y=[pa1.nresidues for value in res_interest],
        orientation="v",
        width=2,
        marker={"color": res_interest, "colorscale": "Viridis_r"},
        opacity=0.6

        )
             )

    fig.update_layout(
        title_text=title,
        width=700,
        height=700
    )

    fig.show()
    
    if save:
        if not os.path.exists("images"):
            os.mkdir("images")
        fig.write_image(f"images/{title}.svg")
        fig.write_image(f"images/{title}.png", )
    # Save filtered array in the same format as CONAN peason_data.dat output, so it can be accessed by PearsonAnalysis() in the same way
    with open(f"images/{title}.dat", "w") as ouf:
        for row in range(0, len(filt_by_cent)):
            for col in range(0, len(filt_by_cent[row])):
                line = f"{row+1}  {col+1}  {filt_by_cent[row][col]}\n"
                ouf.write(line) 
    

In [ ]:
# Average out all CONAN plots that have the openning movement (so excluding 02 and 03) for PearsonAnalysis()

# Get raw CONAN data into a list of lists
sims = ["01", "04", "05", "06", "07", "08", "09", "10"]
raw_list = []
for sim in sims:
    path = f"CONAN/conan1_{sim}/aggregate/pearson_data.dat"
    if os.path.exists(path):
        #print(path)
        pa = PearsonAnalysis(path)
        raw_list.append(pa.raw_data)
print(raw_list)
# Get an array of mean values across the simulations for each array point
mean_array = numpy.zeros((370,370))
for row in range(0, len(mean_array)):
    for col in range(0, len(mean_array[row])):
        mean_array[row][col] = numpy.mean([raw[row][col] for raw in raw_list])
print(mean_array)
# Save mean array in the same format as CONAN peason_data.dat output, so it can be accessed by PearsonAnalysis() in the same way
with open("CONAN/2022-09-13_mbp_all_sim_conans_mean.dat", "w") as ouf:
    for row in range(0, len(mean_array)):
        for col in range(0, len(mean_array[row])):
            line = f"{row+1}  {col+1}  {mean_array[row][col]}\n"
            ouf.write(line) 

In [ ]:
conan_graph("CONAN/2022-09-13_mbp_all_sim_conans_mean.dat", title="Unfiltered mean", save=0)

In [ ]:
conan_graph("CONAN/2022-09-13_mbp_all_sim_conans_mean.dat", meanP=99, medP=99, mxmnP=99, title="MBP_all_>99th", save=0)

# dDIHE

In [ ]:
# Compare the crystal structure dihedrals with the dihedrals of the last 50 ns of the 100 ns apo and the 200 ns holo simulations
# Get the dDIHE. HOLO, last 50 ns of all 10. APO, last 50 ns of all excluding 2 and 3 (in which the opening transition doesn't take place)

import mdtraj as md
import numpy as np
import math
import os

output_dict = {}
state = "apo"
apo_top_path = "Simulations/Apo/1anf_malremoved_t3p.parm7"

for sim in [1,4,5,6,7,8,9,10]:
    print(f"{state} sim {sim}...")
    apo_dcd_path = f"Simulations/Apo/npt_production_{sim:02d}.dcd"
    
    traj = md.load_dcd(apo_dcd_path, top=apo_top_path, stride=5) # Load a frame for every ns of the simulation
    last_50 = traj[-50:] # Trajectory object of the last 50 frames

    calpha_numbers = [a.index for a in traj.topology.atoms if a.name == 'CA'] # For the atoms in topology, make a list of the atoms that are CA
    fours = [calpha_numbers[calpha_numbers.index(at)-1:calpha_numbers.index(at)+3] for at in calpha_numbers[1:-2]] # Get a list of lists of 4 atoms
    all_radians = md.compute_dihedrals(last_50, fours) # Compute radians for all positions at all time points

    for i in range(0, len(fours)): # Each item in fours is a list of four atom indicies. There is an item for each residue that has 1 residue behind and 2 in front
        res = str(traj.topology.atom(fours[i][1]))[0:-3] # The str() contents will be RESXX-CA where RES is the 3 letter code, XX is the residue number, CA is indicating that it is the calpha. res = [0:-3] of this string, getting rid of the "-CA" leaving only "RESXX"

        if res not in output_dict.keys(): # On the first pass, create the dictionary and fill with information that applies to all states:
            output_dict[res] = dict() # Create a dictionary of information for the residue corresponding to this atom
            output_dict[res]["dDIHE_mean_of_means"] = float()

        if state not in output_dict[res].keys(): # Create dictionary for each state (apo or holo)
            output_dict[res][state] = dict()
            output_dict[res][state]["DIHE_means"] = []
            output_dict[res][state]["DIHE_mean_of_means"] = float()

        if f"{sim:02d}" not in output_dict[res][state].keys(): # For each new simulation:
            output_dict[res][state][f"{sim:02d}"] = dict()
            output_dict[res][state][f"{sim:02d}"]["dihe_per_ns"] = [] # The dihedral angle at this position for each nanosecond of this simulation.
            output_dict[res][state][f"{sim:02d}"]["dihe_mean"] = float()
            output_dict[res][state][f"{sim:02d}"]["dihe_stdv"] = float()

        for ns in all_radians:
            output_dict[res][state][f"{sim:02d}"]["dihe_per_ns"].append(ns[i]) # Get the dihedral angles for this simulation at this position for each nanosecond

        output_dict[res][state][f"{sim:02d}"]["dihe_mean"] = np.mean(output_dict[res][state][f"{sim:02d}"]["dihe_per_ns"])
        output_dict[res][state][f"{sim:02d}"]["dihe_stdv"] = np.std(output_dict[res][state][f"{sim:02d}"]["dihe_per_ns"])
        output_dict[res][state]["DIHE_means"].append(output_dict[res][state][f"{sim:02d}"]["dihe_mean"])

state = "holo"
bou_top_path = "Simulations/Holo/1anf_mal_t3p.parm7"

for sim in range(1,11):
    print(f"{state} sim {sim}...")
    bou_dcd_path = f"Simulations/Holo/npt_production_{sim:02d}.dcd"
    traj = md.load_dcd(bou_dcd_path, top=bou_top_path, stride=5) # Load a frame for every ns of the simulation
    last_50 = traj[-50:] # Trajectory object of the last 50 frames
    
    calpha_numbers = [a.index for a in traj.topology.atoms if a.name == 'CA'] # For the atoms in topology, make a list of the atoms that are CA
    fours = [calpha_numbers[calpha_numbers.index(at)-1:calpha_numbers.index(at)+3] for at in calpha_numbers[1:-2]] # Get a list of lists of 4 atoms
    all_radians = md.compute_dihedrals(last_50, fours) # Compute radians for all positions at all time points
    
    for i in range(0, len(fours)): # Each item in fours is a list of four atom indicies. There is an item for each residue that has 1 residue behind and 2 in front
        res = str(traj.topology.atom(fours[i][1]))[0:-3] # The str() contents will be RESXX-CA where RES is the 3 letter code, XX is the residue number, CA is indicating that it is the calpha. res = [0:-3] of this string, getting rid of the "-CA" leaving only "RESXX"

        if res not in output_dict.keys(): # On the first pass, create the dictionary and fill with information that applies to all states:
            output_dict[res] = dict() # Create a dictionary of information for the residue corresponding to this atom
            output_dict[res]["dDIHE_mean_of_means"] = float()

        if state not in output_dict[res].keys(): # Create dictionary for each state (apo or holo)
            output_dict[res][state] = dict()
            output_dict[res][state]["DIHE_means"] = []
            output_dict[res][state]["DIHE_mean_of_means"] = float()

        if f"{sim:02d}" not in output_dict[res][state].keys(): # For each new simulation:
            output_dict[res][state][f"{sim:02d}"] = dict()
            output_dict[res][state][f"{sim:02d}"]["dihe_per_ns"] = [] # The dihedral angle at this position for each nanosecond of this simulation.
            output_dict[res][state][f"{sim:02d}"]["dihe_mean"] = float()
            output_dict[res][state][f"{sim:02d}"]["dihe_stdv"] = float()

        for ns in all_radians:
            output_dict[res][state][f"{sim:02d}"]["dihe_per_ns"].append(ns[i]) # Get the dihedral angles for this simulation at this position for each nanosecond

        output_dict[res][state][f"{sim:02d}"]["dihe_mean"] = np.mean(output_dict[res][state][f"{sim:02d}"]["dihe_per_ns"])
        output_dict[res][state][f"{sim:02d}"]["dihe_stdv"] = np.std(output_dict[res][state][f"{sim:02d}"]["dihe_per_ns"])
        output_dict[res][state]["DIHE_means"].append(output_dict[res][state][f"{sim:02d}"]["dihe_mean"])

for state in ["apo", "holo"]:
    for res in output_dict.keys():
        output_dict[res][state]["DIHE_mean_of_means"] = np.mean(output_dict[res][state]["DIHE_means"])
    
for res in output_dict.keys():    
    change = output_dict[res]["apo"]["DIHE_mean_of_means"] - output_dict[res]["holo"]["DIHE_mean_of_means"]
    
    if -np.pi <= change <= np.pi:
        output_dict[res]["dDIHE_mean_of_means"] = change
    if change > np.pi:
        output_dict[res]["dDIHE_mean_of_means"] = 2*np.pi - (change)
    if change < -np.pi:
        output_dict[res]["dDIHE_mean_of_means"] = 2*np.pi + (change)

# write results to CSV
import csv
    
with open("dDIHE/2022-07-27_maltosePBP_sim_exc2,3_dDIHE.csv", "w", encoding = "latin") as outf:
    csv_columns = ()
    csv_data = [["residue identified", "residue number", "dDIHE"]]
    
    for res in output_dict.keys():
        csv_data.append([]) # want residue identifier, residue number, dDIHE
        csv_data[-1].append(res) # Write a list of lists where each entry is a row
        csv_data[-1].append(res[3:])
        csv_data[-1].append(output_dict[res]["dDIHE_mean_of_means"])
        
        
    writer= csv.writer(outf)
    for i in range(0,len(csv_data)):
        writer.writerow(csv_data[i])



# RMSF

In [ ]:
# Generate dRMSF data

output = {}

import csv
import scipy.stats
import os
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sim = 0

apo_top = "Simulations/Apo/1anf_malremoved_t3p.parm7"
apo_dcd = f"Simulations/Apo/npt_production_{sim:02d}.dcd"    
hol_top = "Simulations/Holo/1anf_mal_t3p.parm7"
hol_dcd = f"Simulations/Holo/simulation_{sim:02d}/npt_production_{sim:02d}.dcd"

state_dict = {"apo": {"top" : apo_top, "dcd" : apo_dcd}, "hol": {"top" : hol_top, "dcd" : hol_dcd}}

output = {}

for state in ["hol", "apo"]:
    output[state] = {}
    print(state)
    if state == "hol":
        for sim in [1,2,3,4,5,6,7,8,9,10]:
            print(sim)
            # Get the paths to the topologies and dcds
            apo_top = "Simulations/Apo/1anf_malremoved_t3p.parm7"
            apo_dcd = f"Simulations/Apo/npt_production_{sim:02d}.dcd"    
            hol_top = "Simulations/Holo/1anf_mal_t3p.parm7"
            hol_dcd = f"Simulations/Holo/simulation_{sim:02d}/npt_production_{sim:02d}.dcd"
            state_dict = {"apo": {"top" : apo_top, "dcd" : apo_dcd}, "hol": {"top" : hol_top, "dcd" : hol_dcd}}

            u = mda.Universe(state_dict[state]["top"],state_dict[state]["dcd"]) # reads as: u = mda.Universe(apo_top,apo_dcd)

            average = align.AverageStructure(u, u, select='protein and name CA',
                                         ref_frame=0).run()
            ref = average.results.universe

            aligner = align.AlignTraj(u, ref,
                                  select='protein and name CA',
                                  in_memory=True).run() # Align the entire trajectory to the reference (average) structure of the trajectory
            print(f"calculating rmsf {state} {sim}")
            c_alphas = u.select_atoms('protein and name CA') # get a c_alphas object from our reference-aligned universe
            R = rms.RMSF(c_alphas).run() # Calculate the RMSF of the c-alphas


            if "per_sim" not in output[state].keys():
                output[state]["per_sim"] = {}
            if f"{sim:02d}" not in output[state]["per_sim"].keys():
                output[state]["per_sim"][f"{sim:02d}"] = R.results.rmsf
        
        # Define the list of residues
        res_list =  [c_alphas.resnames[i]+str(int(c_alphas.resnums[i]-1)) for i in range(0,len(c_alphas.resnames))]
        for res in res_list:
            # Get the rmsf values for this residue across all simulations
            rmsf_per_res = []
            for sim in output[state]["per_sim"].keys():
                rmsf_per_res.append(output[state]["per_sim"][f"{sim}"][res_list.index(res)])
            if "per_res" not in output[state].keys():
                output[state]["per_res"] = {}
            output[state]["per_res"][res] = {}
            output[state]["per_res"][res]["rmsf_values"] = rmsf_per_res
            output[state]["per_res"][res]["mean"] = scipy.stats.tmean(output[state]["per_res"][res]["rmsf_values"])
            output[state]["per_res"][res]["SD"] = scipy.stats.tstd(output[state]["per_res"][res]["rmsf_values"])
            output[state]["per_res"][res]["SEM"] = scipy.stats.sem(output[state]["per_res"][res]["rmsf_values"])

    if state == "apo":
        for sim in [1,4,5,6,7,8,9,10]:
            print(sim)
            # Get the paths to the topologies and dcds
            apo_top = "Simulations/Apo/1anf_malremoved_t3p.parm7"
            apo_dcd = f"Simulations/Apo/npt_production_{sim:02d}.dcd"    
            hol_top = "Simulations/Holo/1anf_mal_t3p.parm7"
            hol_dcd = f"Simulations/Holo/simulation_{sim:02d}/npt_production_{sim:02d}.dcd"
            state_dict = {"apo": {"top" : apo_top, "dcd" : apo_dcd}, "hol": {"top" : hol_top, "dcd" : hol_dcd}}

            u = mda.Universe(state_dict[state]["top"],state_dict[state]["dcd"]) # reads as: u = mda.Universe(apo_top,apo_dcd)

            average = align.AverageStructure(u, u, select='protein and name CA',
                                         ref_frame=0).run()
            ref = average.results.universe

            aligner = align.AlignTraj(u, ref,
                                  select='protein and name CA',
                                  in_memory=True).run() # Align the entire trajectory to the reference (average) structure of the trajectory
            print(f"calculating rmsf {state} {sim}")
            c_alphas = u.select_atoms('protein and name CA') # get a c_alphas object from our reference-aligned universe
            R = rms.RMSF(c_alphas).run() # Calculate the RMSF of the c-alphas


            if "per_sim" not in output[state].keys():
                output[state]["per_sim"] = {}
            if f"{sim:02d}" not in output[state]["per_sim"].keys():
                output[state]["per_sim"][f"{sim:02d}"] = R.results.rmsf
        
        # Define the list of residues
        res_list =  [c_alphas.resnames[i]+str(int(c_alphas.resnums[i]-1)) for i in range(0,len(c_alphas.resnames))]
        for res in res_list:
            # Get the rmsf values for this residue across all simulations
            rmsf_per_res = []
            for sim in output[state]["per_sim"].keys():
                rmsf_per_res.append(output[state]["per_sim"][f"{sim}"][res_list.index(res)])
            if "per_res" not in output[state].keys():
                output[state]["per_res"] = {}
            output[state]["per_res"][res] = {}
            output[state]["per_res"][res]["rmsf_values"] = rmsf_per_res
            output[state]["per_res"][res]["mean"] = scipy.stats.tmean(output[state]["per_res"][res]["rmsf_values"])
            output[state]["per_res"][res]["SD"] = scipy.stats.tstd(output[state]["per_res"][res]["rmsf_values"])
            output[state]["per_res"][res]["SEM"] = scipy.stats.sem(output[state]["per_res"][res]["rmsf_values"])

output["dRMSF"] = {}
for res in res_list:
    output["dRMSF"][res] = output["hol"]["per_res"][res]["mean"] - output["apo"]["per_res"][res]["mean"]

# Record all RMSF data into csvs
for state in state_dict.keys():
    with open(f"dRMSF/2022-08-01_{state}_rmsf_per_residue.csv","w",encoding="latin") as outf:
        writer = csv.writer(outf)
        res_list =  [c_alphas.resnames[i]+str(int(c_alphas.resnums[i]-1)) for i in range(0,len(c_alphas.resnames))]
        headers = res_list
        headers.insert(0,"res_id")
        writer.writerow(headers)
        for sim in output[state]["per_sim"].keys():
            row = list(output[state]["per_sim"][sim])
            row.insert(0,sim)
            writer.writerow(row)
                
# Record dRMSF        
with open("dRMSF/2023-02-08_drmsf_per_residue_excl_apo2,3.csv","w",encoding="latin") as outf:
    header = []
    apo = []
    holo = []
    drmsf = []
    for res in output["dRMSF"].keys():
        header.append(res)
        apo.append(output["apo"]["per_res"][res]["mean"])
        holo.append(output["hol"]["per_res"][res]["mean"])
        drmsf.append(output["dRMSF"][res])
    header.insert(0,"residue")
    apo.insert(0,"apo")
    holo.insert(0,"holo")
    drmsf.insert(0,"dRMSF")
    writer = csv.writer(outf)
    writer.writerow(header)
    writer.writerow(apo)
    writer.writerow(holo)
    writer.writerow(drmsf)